In [6]:
import requests
import pandas as pd
import time
from time import ctime
import pickle
import os

headers = {
    'authority': 'account.divvybikes.com',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': '*/*',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36',
    'content-type': 'application/json',
    'origin': 'https://account.divvybikes.com',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://account.divvybikes.com/map',
    'accept-language': 'en-US,en;q=0.9',
    'cookie': 'sessId=27980fde-5664-40bc-a77e-d336f1bc87e4L1625200883; _ga=GA1.2.22012294.1625200883; OptanonAlertBoxClosed=2021-07-02T04:41:26.613Z; _ga=GA1.3.22012294.1625200883; _gid=GA1.2.4210394.1626280437; _gid=GA1.3.4210394.1626280437; bfe-fpval=0; bfe-utm={}; _gat_UA-17974252-9=1; _gat_UA-1446928-23=1; OptanonConsent=isIABGlobal=false&datestamp=Wed+Jul+14+2021+11%3A54%3A39+GMT-0500+(Central+Daylight+Time)&version=6.4.0&landingPath=NotLandingPage&groups=1%3A1%2C2%3A0%2C3%3A0%2C4%3A0%2C0_286593%3A0%2C0_286594%3A0%2C0_286595%3A0%2C0_286596%3A0&AwaitingReconsent=false',
}

data = '{"operationName":"GetSystemSupply","variables":{},"query":"query GetSystemSupply {\\n supply {\\n stations {\\n stationId\\n stationName\\n location {\\n lat\\n lng\\n __typename\\n }\\n bikesAvailable\\n bikeDocksAvailable\\n ebikesAvailable\\n totalBikesAvailable\\n isValet\\n isOffline\\n isLightweight\\n displayMessages\\n siteId\\n ebikes {\\n batteryStatus {\\n distanceRemaining {\\n value\\n unit\\n __typename\\n }\\n percent\\n __typename\\n }\\n __typename\\n }\\n lastUpdatedMs\\n __typename\\n }\\n rideables {\\n location {\\n lat\\n lng\\n __typename\\n }\\n rideableType\\n batteryStatus {\\n distanceRemaining {\\n value\\n unit\\n __typename\\n }\\n percent\\n __typename\\n }\\n __typename\\n }\\n notices {\\n localizedTitle\\n localizedDescription\\n __typename\\n }\\n requestErrors {\\n localizedTitle\\n localizedDescription\\n __typename\\n }\\n __typename\\n }\\n}\\n"}'

def get_basic_info():
    
    r = requests.post('https://account.divvybikes.com/bikesharefe-gql', headers=headers, data=data).json()
    stations = list(((r['data'])['supply'])['stations'])
    stationNames = []
    
    for i in range(len(stations)):
        stationNames.append(stations[i]['stationName'])
    geotags = []
    
    for i in range(len(stations)):
        geotags.append(stations[i]['location'])

    lats = []
    for i in range(len(geotags)):
        lats.append(geotags[i]['lat'])
     
    longs = []
    for i in range(len(geotags)):
        longs.append(geotags[i]['lng'])
    
    df = pd.DataFrame(index = stationNames)
    
    df['lattitude'] = lats
    df['longitude'] = longs

    return df

def get_more_info(prevdata):
    
    
    #collects data
    r = requests.post('https://account.divvybikes.com/bikesharefe-gql', headers=headers, data=data).json()
    #isolates station data
    stations = list(((r['data'])['supply'])['stations'])
    
    
    
    #gets the names of the station
    stationNames = []
    for i in range(len(stations)):
        stationNames.append(stations[i]['stationName'])
     
    #gets number of bikes at each station
    bikesAvailable = []
    for i in range(len(stations)):
        bikesAvailable.append(stations[i]['totalBikesAvailable'])
    
    #makes a dataframe out of current data
    newdata = pd.DataFrame(index = stationNames)
    newdata[ctime()] = bikesAvailable
    
    prevdata = pd.merge(prevdata,newdata, left_index = True, right_index = True, how = 'outer')
    return prevdata

    #merge prevdata with current data 
#    older code revert back if never version does not work
#     for i in list(prevdata.index):
#         if i in list(df.index):
#             newdata.append(df.loc[i])
#         else:
#             newdata.append(float("nan"))
            
#     prevdata[list(df.columns)] = newdata
    
#     return prevdata


    
        

In [7]:
prev= get_basic_info()
for i in range(180):
    prev = get_more_info(prev)
    prev.to_csv(r'/Users/jakefischer/DivvyScrape/DivvyData/div_data6.csv')
    #sleep for 55 seconds as it takes about 5 seconds to process, so total wait is about 60 seconds
    time.sleep(55)

    

ConnectionError: HTTPSConnectionPool(host='account.divvybikes.com', port=443): Max retries exceeded with url: /bikesharefe-gql (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb5e9d85190>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [4]:
df = pd.read_csv(r'/Users/jakefischer/DivvyScrape/DivvyData/div_data6.csv')